In [1]:
%matplotlib inline

In [2]:
import nltk

In [5]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

In [6]:
sent = "Mary saw Bob".split()

In [7]:
rd_parser = nltk.RecursiveDescentParser(grammar1)

In [8]:
for tree in rd_parser.parse(sent):
     print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [3]:
grammar1 = nltk.data.load('file:mygrammar.cfg')

In [11]:
sent = "Mary saw Bob".split()

In [12]:
rd_parser = nltk.RecursiveDescentParser(grammar1)

In [13]:
for tree in rd_parser.parse(sent):
     print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [14]:
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | V NP | V S | V NP PP
  PP -> P NP
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
  P -> 'on'
  """)

In [4]:
rd_parser = nltk.RecursiveDescentParser(grammar1)

In [5]:
sent = 'Mary saw a dog'.split()

In [6]:
for tree in rd_parser.parse(sent):
    print(tree)
# (S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [7]:
sr_parser = nltk.ShiftReduceParser(grammar1)

In [8]:
sent = 'Mary saw a dog'.split()

In [9]:
for tree in sr_parser.parse(sent):
    print(tree)
#   (S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [10]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [12]:
groucho_grammar = nltk.data.load('file:groucho_grammar.cfg')
groucho_grammar.productions(rhs=text[1])
# [V -> 'shot']

[V -> 'shot']

In [13]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

def display(wfst, tokens):
    print('\nWFST ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

In [14]:
tokens = "I shot an elephant in my pajamas".split()

In [15]:
wfst0 = init_wfst(tokens, groucho_grammar)

In [16]:
display(wfst0, tokens)
# WFST 1    2    3    4    5    6    7
# 0    NP   .    .    .    .    .    .
# 1    .    V    .    .    .    .    .
# 2    .    .    Det  .    .    .    .
# 3    .    .    .    N    .    .    .
# 4    .    .    .    .    P    .    .
# 5    .    .    .    .    .    Det  .
# 6    .    .    .    .    .    .    N


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


In [18]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)

In [19]:
display(wfst1, tokens)
# WFST 1    2    3    4    5    6    7
# 0    NP   .    .    S    .    .    S
# 1    .    V    .    VP   .    .    VP
# 2    .    .    Det  NP   .    .    .
# 3    .    .    .    N    .    .    .
# 4    .    .    .    .    P    .    PP
# 5    .    .    .    .    .    Det  NP
# 6    .    .    .    .    .    .    N


WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


In [20]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)
# [2] Det [3]   N [4] ==> [2]  NP [4]
# [5] Det [6]   N [7] ==> [5]  NP [7]
# [1]   V [2]  NP [4] ==> [1]  VP [4]
# [4]   P [5]  NP [7] ==> [4]  PP [7]
# [0]  NP [1]  VP [4] ==> [0]   S [4]
# [1]  VP [4]  PP [7] ==> [1]  VP [7]
# [0]  NP [1]  VP [7] ==> [0]   S [7]

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
